#Detecting Anomaly

In [ ]:
pip install orion-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 9.3 MB/s 
     |████████████████████████████████| 149 kB 23.0 MB/s 
     |████████████████████████████████| 5.4 MB 26.7 MB/s 
     |████████████████████████████████| 17.3 MB 59.0 MB/s 
     |████████████████████████████████| 2.5 MB 46.6 MB/s 
     |████████████████████████████████| 2.9 MB 38.8 MB/s 
     |████████████████████████████████| 377 kB 52.7 MB/s 
     |████████████████████████████████| 180 kB 57.4 MB/s 
     |████████████████████████████████| 3.1 MB 48.5 MB/s 
     |████████████████████████████████| 10.1 MB 47.6 MB/s 
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
     |████████████████████████████████| 25.4 MB 1.3 MB/s 
     |████████████████████████████████| 110.5 MB 1.5 kB/s 
     |████████████████████████████████| 240 kB 55.7 MB/s 
     |███████████████████████████

In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
from orion import Orion
import yfinance as yf
import pandas as pd
import numpy as np

##Importing Data

In [ ]:
#Importig Apple Stock data from yfinance
start = "2000-01-01"
end = "2020-01-01"

In [ ]:
appl_df = yf.download("AAPL", start=start, end=end)
#Getting Date and Adjusted Closing Price of Apple Stock
appl_df_1 = appl_df.reset_index()[['Date','Adj Close']]
aapl = yf.Ticker('AAPL')

#Making table with Date converted to timestamp in nanoseconds
tmdata = pd.Series(appl_df_1['Date'].values.astype(np.int64))

#Merging table with timestamp with original Table
appl_df_2 = appl_df_1.merge(tmdata.rename('Timestamp'), how = 'inner', left_index= True, right_index = True)

#Dropping Date
appl_df_2.drop('Date', axis=1, inplace= True)

[*********************100%***********************]  1 of 1 completed


In [ ]:
hist = aapl.history(start=start, end=end)
appl_df_3 = appl_df_2.reindex(columns = ['Timestamp', 'Adj Close'])
appl_final = appl_df_3.rename(columns={'Timestamp': 'timestamp', 'Adj Close': 'value'})
appl_final['value'] = appl_final['value'] /appl_final['value'].abs().max()
appl_final['timestamp'] = appl_final['timestamp'].div(1000000000)

##Pipelines

###AER

In [ ]:
orion_aer = Orion(
    pipeline='aer',
)

Using TensorFlow backend.


In [ ]:
orion_aer.fit(appl_final.head(3000))
anomalies_aer = orion_aer.detect(appl_final)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 13844 samples, validate on 3461 samples
Epoch 1/35
13844/13844 [==============================] - 77s 6ms/sample - loss: 0.0290 - tf_op_layer_strided_slice_loss: 0.0331 - tf_op_layer_strided_slice_1_loss: 0.0262 - tf_op_layer_strided_slice_2_loss: 0.0303 - val_loss: 0.1361 - val_tf_op_layer_strided_slice_loss: 0.1324 - val_tf_op_layer_strided_slice_1_loss: 0.1331 - val_tf_op_layer_strided_slice_2_loss: 0.1487
Epoch 2/35
13844/13844 [==============================] - 70s 5ms/sample - loss: 0.0170 - tf_op_layer_strided_slice_loss: 0.0173 - tf_op_layer_strided_slice_1_loss: 0.0168 - tf_op_layer_strided_slice_2_loss: 0.0171 - val_loss: 0.1305 - val_tf_op_layer_strided_slice_loss: 0.1262 - val_tf_op_layer_strided_slice_1_loss: 0.1284 - val_tf_op_layer_strided_slice_2_loss: 0.1346
Epoch 3/35
13844/13844 [==============================] - 70s 5ms/sample - loss: 0.0166 - tf_op_layer_strided_slice_loss: 0.0167 - tf_op_layer_strided_slice_1_loss: 0.0165 - tf_op_layer_strided_slice_2_los

In [ ]:
anomalies_aer

,start,end,severity


###LSTM

In [ ]:
orion_lstm = Orion(
    pipeline='lstm_dynamic_threshold',
)

In [ ]:
orion_lstm.fit(appl_final.head(3000))
anomalies_lstm = orion_lstm.detect(appl_final)

KeyboardInterrupt: ignored

In [ ]:
anomalies_lstm

###TadGAN

In [ ]:
orion_tadgan = Orion(
    pipeline='tadgan'
)

In [ ]:
orion_tadgan.fit(appl_final.head(3000))
anomalies_tadgan = orion_tadgan.detect(appl_final)

In [ ]:
anomalies_tadgan

###Arima

In [ ]:
orion_arima = Orion(
    pipeline='arima'
)


In [ ]:
orion_arima.fit(appl_final.head(3000))
anomalies_arima = orion_arima.detect(appl_final)

In [ ]:
anomalies_arima

##Plot

In [ ]:
#LSTM
anomalies_lstm['start'] = pd.to_datetime(anomalies_lstm['start'], unit='s')
anomalies_lstm['end'] = pd.to_datetime(anomalies_lstm['end'], unit='s')

#TadGAN
anomalies_tadgan['start'] = pd.to_datetime(anomalies_tadgan['start'], unit='s')
anomalies_tadgan['end'] = pd.to_datetime(anomalies_tadgan['end'], unit='s')

#ARIMA
anomalies_arima['start'] = pd.to_datetime(anomalies_arima['start'], unit='s')
anomalies_arima['end'] = pd.to_datetime(anomalies_arima['end'], unit='s')

In [ ]:
anomalies_hist_lstm = [aapl.history(start=anomalies_lstm['start'].iloc[i], end=anomalies_lstm['end'].iloc[i]) for i in range(anomalies_lstm.shape[0])]
anomalies_hist_tadgan = [aapl.history(start=anomalies_tadgan['start'].iloc[i], end=anomalies_tadgan['end'].iloc[i]) for i in range(anomalies_tadgan.shape[0])]
anomalies_hist_arima = [aapl.history(start=anomalies_arima['start'].iloc[i], end=anomalies_arima['end'].iloc[i]) for i in range(anomalies_arima.shape[0])]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
fig_lstm = make_subplots(specs=[[{"secondary_y": True}]])
fig_lstm.add_trace(go.Scatter(x=hist.index,y=hist['Close'],name='Price'),secondary_y=False)
for i in range(len(anomalies_hist_lstm)):
  fig_lstm.add_trace(go.Scatter(x=anomalies_hist_lstm[i].index.values,y=anomalies_hist_lstm[i]['Close'],name='Anomaly' + str(i)),secondary_y=False)
fig_lstm.update_layout(title_text='LSTM')

fig_tadgan = make_subplots(specs=[[{"secondary_y": True}]])
fig_tadgan.add_trace(go.Scatter(x=hist.index,y=hist['Close'],name='Price'),secondary_y=False)
for i in range(len(anomalies_hist_tadgan)):
  fig_tadgan.add_trace(go.Scatter(x=anomalies_hist_tadgan[i].index.values,y=anomalies_hist_tadgan[i]['Close'],name='Anomaly' + str(i)),secondary_y=False)
fig_tadgan.update_layout(title_text='TadGAN')

fig_arima = make_subplots(specs=[[{"secondary_y": True}]])
fig_arima.add_trace(go.Scatter(x=hist.index,y=hist['Close'],name='Price'),secondary_y=False)
for i in range(len(anomalies_hist_arima)):
  fig_arima.add_trace(go.Scatter(x=anomalies_hist_arima[i].index.values,y=anomalies_hist_arima[i]['Close'],name='Anomaly' + str(i)),secondary_y=False)
fig_arima.update_layout(title_text='ARIMA')

###Graph

In [ ]:
fig_lstm.show()
fig_tadgan.show()
fig_arima.show()

#Extracting sentiments from financial op-ed and tweets

##Importing Data

In [ ]:
import pandas as pd
import io
import datetime

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/financial_oped_tweets.csv')

In [ ]:
df=df.dropna(subset=['content'])

In [ ]:
df['category'].unique()

array(['news', 'opinion', 'tweet'], dtype=object)

In [ ]:
anomaly_start = "2018-01-08"
anomaly_end = "2018-01-17"

In [ ]:
df1 = df[ df['ticker'] == 'AAPL' ]
df1 = df1[ df1['release_date'].between(anomaly_start, anomaly_end) ]

In [ ]:
df1.sample(n=5)

,Unnamed: 0,ticker,title,category,content,release_date,writer
113974,113974,AAPL,CME TiVo Texas Instruments STMicroelectroni...,opinion,For Immediate ReleaseChicago IL January 10 ...,2018-01-09,Zacks Investment Research
115065,115065,AAPL,5 Stocks To Fall Back On Amid Lurking Governme...,opinion,The unprecedented reversal in the stock market...,2018-01-16,Zacks Investment Research
114815,114815,AAPL,Comcast Slapped With Patent Infringement Charg...,opinion,Comcast Corp NASDAQ CMCSA the leading ca...,2018-01-15,Zacks Investment Research
115011,115011,AAPL,Should Value Investors Choose Navios Maritime ...,opinion,Value investing is easily one of the most popu...,2018-01-16,Zacks Investment Research
114929,114929,AAPL,Is A WNS Holdings WNS This Earnings Season,opinion,Investors are always looking for stocks that a...,2018-01-16,Zacks Investment Research


##Sentiment Analysis

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
analyzer.polarity_scores('red')

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [ ]:
analyzer.polarity_scores('green')

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

###Ading financial terms to vader_lexicon

In [ ]:
#Replace with dataset
new_words = {
    'red': -1.0,
    'green': 1.0,
}

In [ ]:
analyzer.lexicon.update(new_words)

In [ ]:
analyzer.polarity_scores('green')

{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.25}

###Calc Semantic scores of news title and content

In [ ]:
scores_title = df1['title'].apply(analyzer.polarity_scores).tolist()
scores_content = df1['content'].apply(analyzer.polarity_scores).tolist()
scores_title[0:5]

[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.275, 'neu': 0.22, 'pos': 0.505, 'compound': 0.6369},
 {'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compound': 0.296},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]

In [ ]:
datelist = df1['release_date'].reset_index(drop = True)
df_scores_titles = pd.DataFrame(scores_title)
df_score_content = pd.DataFrame(scores_content)

df_score_content.rename(columns = {'compound':'content_compound'}, inplace = True)
df_scores_titles.rename(columns = {'compound':'titles_compound'}, inplace = True)

compound_sm = pd.merge(df_scores_titles['titles_compound'] , df_score_content['content_compound'] , left_index =True,right_index= True)
compound_sm_np = pd.merge(compound_sm , df1['category'].reset_index(drop=True) , left_index =True,right_index= True)

semantic_oped = pd.merge(compound_sm_np , datelist , left_index =True,right_index= True)
semantic_oped.sample(n=5)

,titles_compound,content_compound,category,release_date
158,0.4019,0.9996,opinion,2018-01-17
144,-0.3818,0.9799,opinion,2018-01-16
160,0.0000,0.9997,opinion,2018-01-17
7,0.0000,0.0000,tweet,2018-01-08
68,0.2500,0.9868,opinion,2018-01-16


##Semantic Factor

Since financial Op-ed contents cotain more information than the titles we give content_compound a more weightage than title_compound in the ratio of 4:1

In [ ]:
semantic_oped['compound'] = 0.2*semantic_oped['titles_compound'] + 0.8*semantic_oped['content_compound']
semantic_oped.sample(n=5)

,titles_compound,content_compound,category,release_date,compound
12,0.0,0.4939,tweet,2018-01-08,0.39512
122,0.0,0.9633,opinion,2018-01-16,0.77064
69,0.0,0.9882,opinion,2018-01-16,0.79056
135,0.0,0.9941,opinion,2018-01-16,0.79528
82,0.0,0.9892,opinion,2018-01-16,0.79136


In [ ]:
semantic_oped2 = semantic_oped
semantic_oped2. drop('titles_compound', axis=1, inplace=True)
semantic_oped2. drop('content_compound', axis=1, inplace=True)
semantic_oped2.sample(n=5)

,category,release_date,compound
76,opinion,2018-01-16,0.86384
30,news,2018-01-10,0.52208
131,opinion,2018-01-16,0.87292
65,news,2018-01-16,0.73792
78,opinion,2018-01-16,0.77600


since all financial statements do not contain the same value we weight all compound values according to the category it belongs to with the order of preference being:

1.   news
2.   opinion
3.   tweet



In [ ]:
news_coeffecient = 0.5
opinion_coeffecient = 0.3
tweet_coeffecient = 0.2

In [ ]:
#category news
news_compound_score = semantic_oped2[ semantic_oped2['category'] == 'news' ].groupby(by=['release_date']).sum()
news_compound_count = semantic_oped2[ semantic_oped2['category'] == 'news' ].groupby(by=['release_date']).count()
news_compound_score = news_compound_score.div(news_compound_count['compound'], axis = 0)

news_compound_score.rename(columns = {'compound':'compound_x'}, inplace = True)

#category opinion
opinion_compound_score = semantic_oped2[ semantic_oped2['category'] == 'opinion' ].groupby(by=['release_date']).sum()
opinion_compound_count = semantic_oped2[ semantic_oped2['category'] == 'opinion' ].groupby(by=['release_date']).count()
opinion_compound_score = opinion_compound_score.div(opinion_compound_count['compound'], axis = 0)

opinion_compound_score.rename(columns = {'compound':'compound_y'}, inplace = True)

#category tweet
tweet_compound_score = semantic_oped2[ semantic_oped2['category'] == 'tweet' ].groupby(by=['release_date']).sum()
tweet_compound_count = semantic_oped2[ semantic_oped2['category'] == 'tweet' ].groupby(by=['release_date']).count()
tweet_compound_score = tweet_compound_score.div(tweet_compound_count['compound'], axis = 0)

tweet_compound_score.rename(columns = {'compound':'compound_z'}, inplace = True)

In [ ]:
ds = pd.concat([news_compound_score, opinion_compound_score,tweet_compound_score], axis=1)
ds = ds.fillna(0)

ds['semantic_factor'] = news_coeffecient*ds['compound_x'] + opinion_coeffecient*ds['compound_y'] + tweet_coeffecient*ds['compound_z']
ds2 = ds

ds

,compound_x,compound_y,compound_z,semantic_factor
release_date,,,,
2018-01-08,0.425380,0.863760,0.171267,0.506071
2018-01-09,0.505465,0.829220,0.000000,0.501499
2018-01-10,0.506756,0.799280,0.000000,0.493162
2018-01-11,0.063600,0.797227,0.019976,0.274963
2018-01-12,0.685707,0.046440,-0.457520,0.265281
2018-01-14,0.601800,0.832493,0.000000,0.550648
2018-01-15,-0.720750,0.806873,0.000000,-0.118313
2018-01-16,0.765660,0.773535,0.000000,0.614891
2018-01-17,0.465185,0.721438,0.000000,0.449024


##Getting points with highest and lowest disturbance

In [ ]:
ds2['semantic_factor'] = ds2['semantic_factor'].abs()

## Get Max 3
ds2 = ds2.sort_values('semantic_factor')
ds2 = ds2[ ds2['semantic_factor'] != 0.0 ]
final_df_h = ds2.tail(3)
final_df_l = ds2.head(3)

final_df_h = final_df_h.sort_index()
final_df_l = final_df_l.sort_index()
final_df_l

,compound_x,compound_y,compound_z,semantic_factor
release_date,,,,
2018-01-11,0.063600,0.797227,0.019976,0.274963
2018-01-12,0.685707,0.046440,-0.457520,0.265281
2018-01-15,-0.720750,0.806873,0.000000,0.118313


In [ ]:
final_news = []
for i in range(len(final_df_h.index)):
  temp = df1[df1['release_date'] == final_df_h.index.values[i]]
  final_news.append(temp)
final_news = pd.concat(final_news)
final_news[['release_date','category','title','content']]

,release_date,category,title,content
113719,2018-01-08,news,Apple should address youth phone addiction sa...,This story corrects Reuters Instrument Code f...
113722,2018-01-08,news,French prosecutor launches probe into Apple pl...,PARIS Reuters A French prosecutor has laun...
113723,2018-01-08,news,Samsung Profit Misses Estimates as Stronger Wo...,Bloomberg Samsung Electronics KS 005930 ...
113724,2018-01-08,news,European shares hit highest since August 2015 ...,By Danilo Masoni MILAN Reuters European sh...
113733,2018-01-08,news,Fairpointe adds to shareholder demands over Nu...,By Liana B Baker Reuters Fairpointe Capit...
...,...,...,...,...
115065,2018-01-16,opinion,5 Stocks To Fall Back On Amid Lurking Governme...,The unprecedented reversal in the stock market...
115067,2018-01-16,opinion,Simon Starbucks Reach Settlement Over Teavana...,Retail REIT Simon Property Group Inc NYSE S...
115068,2018-01-16,opinion,BD s CE Mark And Check Point s Screening Test ...,Becton Dickinson and Company NYSE BDX po...
115069,2018-01-16,opinion,ONE Gas Updates Investors With Near Long Ter...,ONE Gas Inc NYSE OGS revised 2017 guidan...


In [ ]:
hist_anom = aapl.history(start=anomaly_start, end=anomaly_end)
hist_anom

,Open,High,Low,Close,Volume,Dividends,Stock Splits,release_date
Date,,,,,,,,
2018-01-08,41.516102,41.816131,41.416089,41.516102,82271200,0,0,2018-01-08
2018-01-09,41.563728,41.685167,41.292272,41.511341,86336000,0,0,2018-01-09
2018-01-10,41.232736,41.504192,41.194636,41.501808,95839600,0,0,2018-01-10
2018-01-11,41.573250,41.787559,41.549440,41.737553,74670800,0,0,2018-01-11
2018-01-12,41.951855,42.232838,41.825652,42.168545,101672400,0,0,2018-01-12
2018-01-16,42.361429,42.716228,41.942340,41.954247,118263600,0,0,2018-01-16


In [ ]:
final_df_h['release_date'] = final_df_h.index.values

In [ ]:
pd.merge(final_df_h, hist_anom, on='release_date', how='inner')

ValueError: ignored

In [ ]:
cnp_H = pd.merge(final_df_h,hist_anom,right_index=True,left_index=True,how='inner')
cnp_L = pd.merge(final_df_l,hist_anom,right_index=True,left_index=True,how='inner')
cnp_H

,compound_x,compound_y,compound_z,semantic_factor,Open,High,Low,Close,Volume,Dividends,Stock Splits,release_date
